In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install librosa

In [0]:
import librosa
import pickle
import math

In [0]:
import numpy as np
import tensorflow as tf

# Read and transform data

Convert raw byte stream into useful feature space. STFT on the signals serves the initial feature extraction process

In [0]:
base_path = './drive/My Drive/Colab Notebooks/hw4/speaker-verification/'
train_path = base_path + 'hw4_trs.pkl'
test_path = base_path + 'hw4_tes.pkl'

In [0]:
with open(train_path, 'rb') as f:
  train_raw = pickle.load(f)

with open(test_path, 'rb') as f:
  test_raw = pickle.load(f)

trs.pkl contains an 500×16,180 matrix, whose row is a speech signal with 16,180 samples. They are
the returned vectors from the librosa.load function. Similarly, tes.pkl holds a 200×22,631 matrix

In [7]:
train_raw.shape, test_raw.shape

((500, 16180), (200, 22631))

## Convert into audio (.wav file) for checking manually

In [0]:
# for i in range(train_raw.shape[0]):
#   librosa.output.write_wav(base_path + 'trs/trs' + ('0000' + str(i))[-4:] + '.wav', train_raw[i], 16000)

In [0]:
# for i in range(test_raw.shape[0]):
#   librosa.output.write_wav(base_path + 'tes/tes' + ('0000' + str(i))[-4:] + '.wav', test_raw[i], 16000)

## Extract features using STFT

In [0]:
train_complex = np.array([librosa.stft(x, n_fft=1024, hop_length=512).T for x in train_raw])
test_complex = np.array([librosa.stft(x, n_fft=1024, hop_length=512).T for x in test_raw])

In [0]:
train = np.abs(train_complex)
test = np.abs(test_complex)

In [35]:
train.shape, test.shape

((500, 32, 513), (200, 45, 513))

# Generate mini-batches

## Structure of training data

The training matrix is ordered by speakers. Each speaker has 10 utterances, and there are 50 such
speakers (that’s why there are 500 rows). Similarly, the test set has 20 speakers, each of which is with
10 utterances.

## Procedure to generate mini-batches

1. Randomly sample L pairs of utterances from the ten utterance of the first speaker. In theory, there are $10 \choose 2$= 45 pairs you can sample from. You can use all 45 of them if you want. These are the positive examples in your first minibatch

2. Randomly sample L utterances from the 49 training speakers. Using them and the ten utterances of
the first speaker, form another set of L pairs. If L > 10, you’ll need to repeatedly use the first speaker’s
utterance (i.e. sampling with replacement). This set is your negative examples, each of whose pair
contains an utterance from the first speaker and a random utterance spoken by a different speaker.

3. In this first minibatch, you have 2L pairs of utterances.

4. Repeat this process for the other training speakers, so that each speaker is represented by L positive
pairs and L negative pairs. By doing so, you can form 50 minibatches with a balanced number of
positive and negative pair

In [0]:
'''
batch_size: return batch_size number of positive and another batch_size number of negative pairs
stick: if true, one training class will be common across positive and negative pairs

returns:
x: list of pairs
y: 1 for positive pairs, 0 for negative
'''
def next_batch(batch_size, stick=False):
  x1, x2, y = [], [], []
  
  base = np.random.randint(len(train)//10)
  
  # generate positive pairs
  for _ in range(batch_size):
    # randomly select idx_0 only if stick is false
    if not stick:
      base = np.random.randint(len(train)//10)
      
    idx_0, idx_1 = base * 10 + np.random.choice(np.arange(10), size=2, replace=False)
    print(train[idx_0])
    x1.append(train[idx_0])
    x1.append(train[idx_1])
    y.append(1)
#     print(idx_0, idx_1, 1)
    
  # generate negative pairs
  for _ in range(batch_size):
    # randomly select idx_0 only if stick is false
    if not stick:
      base = np.random.randint(len(train)//10)
    
    # make sure neg_base is not same as base
    while True:
      neg_base = np.random.randint(len(train)//10)
      if neg_base != base:
        break
      
    idx_0 = base * 10 + np.random.randint(10)
    idx_1 = neg_base * 10 + np.random.randint(10)
    
    x2.append(train[idx_0])
    x2.append(train[idx_1])
    y.append(0)
#     print(idx_0, idx_1, 0)
    
  return np.array(x1), np.array(x2), np.array(y)

In [31]:
x1, x2, y = next_batch(1)

[[ 7.8817662e-03+0.0000000e+00j -6.6777826e-03-9.8255822e-20j
   4.2955857e-03-7.9463299e-20j ... -2.0995343e-03+7.9463299e-20j
   1.5495003e-03-6.4374504e-20j -9.2036935e-04+0.0000000e+00j]
 [ 1.9520905e-02+0.0000000e+00j -1.2587999e-02+9.1156159e-03j
   2.9805333e-03-7.8195985e-03j ... -1.8303633e-04+4.0014187e-04j
  -1.8029427e-04+2.2568848e-04j  4.5190667e-04+0.0000000e+00j]
 [ 1.8323041e-03+0.0000000e+00j  6.0355542e-03-6.1403280e-03j
  -7.1416842e-03+3.6027331e-03j ... -7.2944364e-05-1.1763087e-03j
  -9.4011681e-05+3.7077026e-04j  1.0530289e-04+0.0000000e+00j]
 ...
 [ 1.6067585e-02+0.0000000e+00j -1.1934178e-02+1.3117272e-02j
   6.7827711e-03+8.2767126e-04j ... -2.2005970e-02+2.4064856e-03j
  -3.7754679e-03-2.6681589e-02j  2.4162024e-02+0.0000000e+00j]
 [ 1.5797660e-02+0.0000000e+00j -3.2998536e-02-2.2043161e-02j
   1.6168015e-02-1.7199272e-02j ... -3.6086276e-02-6.0416840e-02j
   2.3701103e-01+3.5376984e-01j -5.7206970e-01+0.0000000e+00j]
 [ 8.7489029e-03+0.0000000e+00j -3.98638

# Create model

In [0]:
max_length = 50
num_features = 513
num_hidden = 256

learning_rate = 0.001

In [0]:
# Using this implementation as reference https://github.com/ardiya/siamesenetwork-tensorflow/blob/master/train.py

def model(inp, reuse):
  with tf.name_scope('model'):
    
    with tf.variable_scope('lstm_1') as scope:
      cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_hidden, reuse=reuse))
      output, state = tf.nn.dynamic_rnn(cell, inp, dtype=tf.float32)
      
#     with tf.variable_scope('conv_1') as scope:
#       conv_1 = tf.contrib.layers.conv2d(inp, 16, [1, 3], activation_fn=tf.nn.relu, padding='same', \
# 		        weights_initializer=tf.contrib.layers.xavier_initializer_conv2d(), scope=scope, reuse=reuse)
#       pool_1 = tf.contrib.layers.max_pool2d(conv_1, [1, 2], stride=[1, 2])
#       flattened_1 = tf.contrib.layers.flatten(pool_1)
#       dropout_1 = tf.contrib.layers.dropout(flattened_1, keep_prob=0.9)
      
    with tf.variable_scope('dense_1') as scope:
      dense_1 = tf.contrib.layers.fully_connected(output, 513, activation_fn=tf.nn.sigmoid, \
            weights_initializer=tf.contrib.layers.xavier_initializer(), scope=scope, reuse=reuse)
  return dense_1

In [0]:
def contrastive_loss(model1, model2, label, margin):
  with tf.name_scope('contrastive-loss'):
    distance = tf.sqrt(tf.reduce_sum(tf.pow(model1 - model2, 2), 1, keepdims=True))
    similarity = label * tf.square(distance)
    dissimilarity = (1 - label) * tf.square(tf.maximum((margin - distance), 0))
  return tf.reduce_mean(dissimilarity + similarity) / 2

In [0]:
X1 = tf.placeholder(dtype='float', shape=[None, max_length, num_features])
X2 = tf.placeholder(dtype='float', shape=[None, max_length, num_features])
y = tf.placeholder(dtype='float', shape=[None])
margin = 0.2

In [17]:
model1 = model(X1, False)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
model2 = model(X2, True)

In [0]:
loss = contrastive_loss(model1, model2, y, margin)

In [20]:
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss=loss)

Instructions for updating:
Use tf.cast instead.


In [0]:
batch_size = 10
num_epochs = 1
display_step = 1
save_step = 1
log_step = 1

num_samples_tr = train.shape[0]
num_samples_te = test.shape[0]
num_batches_tr = int(math.ceil(num_samples_tr/batch_size))
num_batches_te = int(math.ceil(num_samples_te/batch_size))

train_loss = []
test_loss = []

In [0]:
sess = tf.Session()
saver = tf.train.Saver()

In [23]:
sess.run(tf.global_variables_initializer())

# saver.restore(sess, base_path_model + 'model_95.ckpt')

for epoch in range(num_epochs):
  loss_val = 0
  for i in range(0, num_samples_tr, batch_size):
    batch_x1, batch_x2, batch_y = next_batch(batch_size)
    
    _, lv = sess.run([train_op, loss], feed_dict={X1: batch_x1, X2: batch_x2, y: batch_y})
    loss_val += lv
    
  if epoch % display_step == 0:
    print(epoch, loss_val)
  if epoch % save_step == 0:
    saver.save(sess, base_path_model + 'model_' + str(epoch) + '.ckpt')

163 160 1
470 478 1
20 23 1
38 36 1
191 190 1
329 325 1
439 435 1
323 327 1
292 295 1
279 274 1
464 300 0
176 252 0
19 160 0
74 238 0
469 416 0
300 108 0
459 86 0
110 424 0
498 46 0
41 139 0


/usr/local/lib/python3.6/dist-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


ValueError: ignored

In [0]:

x1, x2, y = next_batch(10)

In [0]:
x1.shape, x2.shape, y.shape